In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
from tqdm import tqdm
import time
import os

In [ ]:
# prendi le recensioni per ogni utente
# dato uno steam id
# è possibile accedere alle recensioni scritte dall'utente tramite la pagina
# https://steamcommunity.com/profiles/{steam_id}/recommended/?p={page_number}
# non tutte le recensioni sono presenti nella stessa pagina
# ma è possibile scorrere le pagine aggiungendo al link
# /?p={page_number}
# (nella prima pagina non è necessario ma si può accedere tranquillamente mettendo p=1)
# il problema è che è possibile accedere anche a pagine dove non sono presenti recensioni
# ma parsando la pagina la lista delle recensioni sarà vuota

In [ ]:
KEY = '0BAEAA44F5C3C9967C53E41AE9B50F25'

In [ ]:
df = pd.read_csv('output_files/games.csv')
df_tmp = df[df['total_recommendations']>0]
app_ids = df_tmp['steam_appid'].tolist()

In [ ]:
players = []
for app_id in tqdm(app_ids, total=len(app_ids)):
    url = f'https://store.steampowered.com/appreviews/{app_id}?json=1&language=all&num_per_page=100'
    response = requests.get(url)
    data = response.json()
    players.extend([review['author']['steamid'] for review in data['reviews']])

In [ ]:
steam_ids = set(players)
# sottrai dall'insieme steam_ids gli steam_id che sono presenti nel file resolved_steam_ids.csv
# in modo da non ripetere le richieste
if os.path.exists('output_files/resolved_steam_ids.csv'):
    print('Already resolved some ids, removing them from the list...')
    df = pd.read_csv('output_files/resolved_steam_ids.csv')
    resolved_steam_ids = set(df['steam_id'].tolist())
    steam_ids = steam_ids - resolved_steam_ids
else:
    print('No resolved ids found.')

In [ ]:
def get_reviews(steam_id, page_number):
    url = f"https://steamcommunity.com/profiles/{steam_id}/recommended/?p={page_number}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    reviews = soup.findAll('div', {'class': 'review_box'})
    return reviews

In [ ]:
with open('output_files/reviews/game_reviews.csv', 'w') as F:
    writer = csv.writer(F)
    cols = ['steam_id', 'app_id', 'recommended', 'hours_played', ]
    writer.writerow(cols)

    for steam_id in tqdm(steam_ids, total=len(steam_ids)):
        try:
            i = 1
            while i!=0:
                reviews = get_reviews(steam_id, i)
                if reviews != []:
                    for review in reviews:
                        review_data = []
                        review_data.append(str(steam_id))
                        app_id = review.find('a', {'class': 'game_capsule_ctn'})['href'][31:]
                        review_data.append(app_id)
                        recommended = review.find('div', {'class': 'title'}).text
                        review_data.append(recommended)
                        hours_played = review.find('div', {'class': 'hours'}).text
                        hours_played = hours_played.replace('\r', '')
                        hours_played = hours_played.replace('\n', '')
                        hours_played = hours_played.replace('\t', '')
                        review_data.append(hours_played)
                        writer.writerow(review_data)
                    i += 1
                    time.sleep(1.5)
                else:
                    i = 0
                    with open('output_files/reviews/resolved_steam_ids.csv', 'a') as F:
                        writer = csv.writer(F)
                        writer.writerow([steam_id])
        except:
            with open('output_files/reviews/error_steam_ids.csv', 'a') as F:
                writer = csv.writer(F)
                writer.writerow([steam_id])